In [4]:
import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
from numba import jit
%matplotlib inline
rcParams['font.size'] = 16

In [2]:
# problem parameters
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

l1_target = 1e-6

u = 1

def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

In [ ]:
# function for omega
def solve_omega(w,l1norm):
    """Iteratively solves the Laplace equation using the Jacobi method
    
    Parameters:
    -----------
    p: 2D array of float
        Initial potential distribution
    l2_target: float
        target for the difference between consecutive solutions
        
    Returns:
    --------
    p: 2D array of float
        Potential distribution after relaxation
    """
    l1norm = 1
    wn = numpy.empty_like(w)
    iterations = 0
    while l1norm > l1_target:
        wn = w.copy()
        w[1:-1,1:-1] = 0.25*(wn[1:-1,2:] + wn[1:-1, :-2] \
                            + wn[2:,1:-1] + wn[:-2,1:-1])
        
        # Neumann B.C. along x=L
        p[1:-1,-1] = p[1:-1,-2]
        l2norm = L2_error(p,pn)
        iterations += 1
    print('Converged in {} iterations'.format(iterations))
    return w



# function for ksi